In [8]:
# 对比算法 maoer_user_pay_pred_model_BiLSTM  “A Prediction Method of User Purchase Behavior Based on Bidirectional Long Short -Term Memory Neural Network Model”

import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import math
import copy
from torch.utils.data import Dataset, DataLoader, TensorDataset
import datetime
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, recall_score, precision_score, roc_curve, confusion_matrix
from _collections import OrderedDict  # 导入 OrderedDict 来保持字典中键值对的顺序

print('||--------开始时间：',datetime.datetime.now(),'-------------')
# data input
data_time_windows_list =['0101_0131','0115_0215','0201_0230','1101_1130','1115_1215','1201_1231','1215_0115']
# data_time_windows = '0101_0131'
# path = './Dataset/' + data_time_windows + '_user_pay_pred_feature_deal.csv'
# dataset_spilt_path = './Dataset/' + data_time_windows + '_user_pay_pred_feature_spilt_BiLSTM.csv'
# # output_weight_result_path = './Dataset/' + data_time_windows + '_user_pay_pred_result_weight.csv'
# data_feature_continue_discrete_namelist_path = './Dataset/maoer_timewindows_continue_discrete_feature_column.csv'    # 连续与离散划分表

# 参数设置
train_ratio = 0.6
val_ratio = 0.2
test_ratio = 0.2
num_heads = 10
feature_dim = 200
max_history_len = 15
num_experts = 3
num_tasks = 2
# 设置嵌入维度
continue_embedding_dim = 200
discrete_embedding_dim = 200
lr = 0.1
batch_size = 128
threshold = 0.5


# 获取时间窗内连续与离散特征名的列表
def get_continue_discrete_feature_namelist(time_windows, datapath):
    data = pd.read_csv(datapath)
    time_windows_data = data[(data['DataSet'] == time_windows)]
    user_history_pay_QOE_continue_column = eval([time_windows_data['QOE_continue'].values.tolist()][0][0])
    user_history_pay_CHONGHE_continue_column = eval([time_windows_data['CHONGHE_continue'].values.tolist()][0][0])
    user_history_pay_FUFEI_continue_column = eval([time_windows_data['FUFEI_continue'].values.tolist()][0][0])
    user_history_pay_QOE_discrete_column = eval([time_windows_data['QOE_discrete'].values.tolist()][0][0])
    user_history_pay_CHONGHE_discrete_column = eval([time_windows_data['CHONGHE_discrete'].values.tolist()][0][0])
    user_history_pay_FUFEI_discrete_column = eval([time_windows_data['FUFEI_discrete'].values.tolist()][0][0])


    return user_history_pay_QOE_continue_column, user_history_pay_CHONGHE_continue_column,user_history_pay_FUFEI_continue_column,\
            user_history_pay_QOE_discrete_column,user_history_pay_CHONGHE_discrete_column,user_history_pay_FUFEI_discrete_column

# # 获取时间窗内连续与离散特征名的列表
# user_history_pay_QOE_continue_column, user_history_pay_CHONGHE_continue_column, \
#         user_history_pay_FUFEI_continue_column, user_history_pay_QOE_discrete_column,\
#         user_history_pay_CHONGHE_discrete_column, user_history_pay_FUFEI_discrete_column = get_continue_discrete_feature_namelist(data_time_windows, data_feature_continue_discrete_namelist_path)
# user_feature_continue_column = []
# user_feature_discrete_column = []

# # total continue feature
# total_continue_feature = user_feature_continue_column+user_history_pay_QOE_continue_column+user_history_pay_CHONGHE_continue_column+user_history_pay_FUFEI_continue_column
# total_discrete_feature = user_feature_discrete_column+user_history_pay_QOE_discrete_column+user_history_pay_CHONGHE_discrete_column+user_history_pay_FUFEI_discrete_column
# total_discrete_feature_add_D = user_feature_discrete_column+user_history_pay_QOE_discrete_column+user_history_pay_CHONGHE_discrete_column+user_history_pay_FUFEI_discrete_column
# total_discrete_feature_add_D.append('user_in_drama_is_pay_for_drama_in_next_time')
# D = 'user_in_drama_is_pay_for_drama_in_next_time'
# tensor_dict_idx = ['pay_QOE_continue','pay_QOE_discrete','pay_CHONGHE_continue','pay_CHONGHE_discrete','pay_FUFEI_continue','pay_FUFEI_discrete','target_QOE_continue','target_QOE_discrete','target_CHONGHE_continue','target_CHONGHE_discrete','target_FUFEI_continue','target_FUFEI_discrete']
# # print(len(user_history_pay_QOE_continue_column),len(user_history_pay_CHONGHE_continue_column),len(user_history_pay_FUFEI_continue_column))
# # 形成对应需要的特征名称列表
# feature_column_dict = {
#     'total_continue_feature': total_continue_feature,
#     'total_discrete_feature': total_discrete_feature,
#     'D':D
# }


||--------开始时间： 2024-03-18 19:21:18.044649 -------------


In [9]:
# 1.数据处理+划分训练、验证、测试集

# 划分数据集 给定输出后固定结果 输出形式定为存储user_id 形成train_dataset,val_dataset,test_dataset
def split_data_unique(input_file, output_file, train_ratio, val_ratio, test_ratio):
    df = pd.read_csv(input_file)
#     data = df[df.columns[0]].unique()  # 提取第一列数据并去重
    data = np.arange(df.shape[0])  #一个从 0 到行数减 1 的整数数组

    np.random.shuffle(data)  # 随机打乱数据
    # 划分数据
    total_len = len(data)
    x_end = int(total_len * train_ratio)
    y_end = x_end + int(total_len * val_ratio)
    train_data = data[:x_end]
    val_data = data[x_end:y_end]
    test_data = data[y_end:]
    # 存储结果是去重的user_id
    result = {
        'Train': train_data,
        'Val': val_data,
        'Test': test_data
    }   
    # 创建每个子集的DataFrame  
    train_df = pd.DataFrame(train_data, columns=['Train'])
    val_df = pd.DataFrame(val_data, columns=['Val'])
    test_df = pd.DataFrame(test_data, columns=['Test'])
    # 将每个DataFrame转换为一列Series  
    train_series = train_df.iloc[:, 0]
    val_series = val_df.iloc[:, 0]
    test_series = test_df.iloc[:, 0]
    # 为了确保所有Series有相同的长度，我们需要找到最大长度并截断较短的Series  
    max_len = max(len(train_series), len(val_series), len(test_series))
    train_series = train_series.reindex(range(max_len)).fillna(value=pd.NA)
    val_series = val_series.reindex(range(max_len)).fillna(value=pd.NA)
    test_series = test_series.reindex(range(max_len)).fillna(value=pd.NA)
    # 创建一个新的DataFrame，将Series作为列  
    combined_df = pd.DataFrame({
        'Train': train_series,
        'Val': val_series,
        'Test': test_series
    })
    # 写入CSV文件，不包含索引和列名  
    combined_df.to_csv(output_file, index=False)
    print('已输出划分数据集结果')

# 数据预处理 将连续特征变离散特征 分桶 不处理user_id、sound_id、drama_id、time
def data_pre_deal(input_path,continue_feature_list):
    df = pd.read_csv(input_path)
    deal_data_df = [] # 待修改********
    # # 获取离散特征的类别数量，并存储为字典
    # category_counts = {}
    # for column in deal_data_df.columns:
    #     unique_values = deal_data_df[column].nunique()  # 获取列的唯一值数量
    #     category_counts[column] = unique_values
    print('数据预处理结束')
    return df

# 根据划分好的数据集中user_id 找到对应csv文件中对应user_id的所有行数据取出，即包含了历史数据（付费+非付费）+目标数据（最后一次行为）
# def find_data_by_list(user_list, intput_data_df, data_hash):
#     df = intput_data_df
#     # result_list = []
#     # 遍历列表中的值，在CSV文件中找到所有匹配的行数据并加入结果列表
#     for user_id in user_list:
#         result_df = df[df[df.columns[0]] == user_id]
#         # result_list.append(result_df)
#         if user_id in data_hash:
#             data_hash[user_id].update({col: result_df for col in df.columns})  # 使用列名作为键
#         else:
#             data_hash[user_id] = {col: result_df for col in df.columns}
#     #result = pd.concat(result_list)  # 合并所有匹配的行数据
#     return data_hash

# 改过 根据 BiLSTM单独改的 根据下标获取数据
def find_data_by_list(index_list, intput_data_df, data_hash):  
    for index in index_list:  
        index = int(index)
        index_list = []
        index_list.append(index)
        result_df = intput_data_df.iloc[index_list]  
        data_hash[index] = result_df  # 直接存储DataFrame对象  
        if result_df.shape[0]==0:
            print('result_df出现空')
    return data_hash
    
# 获取列唯一值数量表，并对离散特征的值转化为从0开始的索引
def get_unique_feature_num_and_discrete_valueChange(datadf,discrete_feature_column_list):
    # 获取离散特征的类别数量，并存储为字典
    feature_category_num_dict = {}
    for column in datadf.columns:
        unique_values_len = datadf[column].nunique()  # 获取列的唯一值数量
        feature_category_num_dict[column] = unique_values_len
        if column in discrete_feature_column_list:
            unique_values = datadf[column].unique()
            value_mapping_dict = {value: index for index, value in enumerate(unique_values) if
                              value != -1 and value != '' and value is not None}
            datadf[column] = datadf[column].map(value_mapping_dict)
    return feature_category_num_dict,datadf

# 总的特征输入，生成划分后数据集及其输入
def data_input(data_time_windows, path, spilt_outpath, train_ratio, val_ratio, test_ratio, total_continue_feature):
    dataset_path = path  # 待修改********
    dataset_spilt_path = spilt_outpath  # 待修改********
    if os.path.exists(dataset_spilt_path):  # 划分训练、验证、测试集
        print("划分文件已存在，不再进行数据划分")
    else:
        split_data_unique(dataset_path, dataset_spilt_path, train_ratio, val_ratio, test_ratio)
    deal_data_df = data_pre_deal(dataset_path, total_continue_feature)  # 数据预处理
    # 获取离散特征的类别数量，并存储为字典
    feature_category_num_dict,deal_data_df  = get_unique_feature_num_and_discrete_valueChange(deal_data_df,total_discrete_feature)
    # 读取划分文件的结果
    spilt_data_df = pd.read_csv(dataset_spilt_path)
    # 输出每一列数据为列表
    train_list = spilt_data_df['Train'].tolist()
    val_list = spilt_data_df['Val'].tolist()
    test_list = spilt_data_df['Test'].tolist()
    train_list = [x for x in train_list if not math.isnan(x)]
    val_list = [x for x in val_list if not math.isnan(x)]
    test_list = [x for x in test_list if not math.isnan(x)]
    # print('训练集、验证集、测试集大小=', len(train_list),len(val_list),len(test_list))
    # 根据划分好的生成以user_id为key的hash（特征集合）将最后一行看做目标数据
    data_hash = {}  # 存成一个hash形式
    find_data_by_list(train_list, deal_data_df, data_hash)
    find_data_by_list(val_list, deal_data_df, data_hash)
    find_data_by_list(test_list, deal_data_df, data_hash)
    print('数据划分完成')
    # print(feature_category_num_dict)
    return train_list, val_list, test_list, data_hash, feature_category_num_dict

# test
# 数据集 train、val、test划分及总数据hash表(以user_id为key的存储对应对应行的hash表)及不同类特征数存储的字典
# train_list, val_list, test_list, data_hash, feature_category_num_dict = data_input(data_time_windows, path,dataset_spilt_path, train_ratio, val_ratio, test_ratio, total_continue_feature)
# print(data_hash[3617476])
# print(feature_category_num_dict)


In [10]:
# 2. 形成张量矩阵 目标特征为：（batch,1,feature_num; 用户历史行为特征为（batch,max_history_len(固定长度的历史记录数),feature_num）

# mask 对用户历史行为长度的固定
# 转换 history 列为长度为max_history_len的数组
def process_history(history, max_history_len):
    if len(history) >= max_history_len:
        processed_history = history[-max_history_len:]
    else:
        processed_history = [-1] * (max_history_len - len(history)) + history
    return processed_history
# 将填充-1的位置标记为True
def create_mask(history):
    mask = [True if item == -1 else False for item in history]
    return mask
# 将历史行为记录处理为固定长度并进行mask
def history_feature_mask(user_history_feature_index, data_matrix_user_history, max_history_len):
    mask_history_feature_matrix = []
    origin_history_feature_matrix = []
    for feature_index in range(len(user_history_feature_index)):
        feature_data = [data_row[feature_index] for data_row in data_matrix_user_history]  # 获取一列特征值
        processed_feature_data = process_history(feature_data, max_history_len)  # 处理为固定长度 max_history_len
        origin_history_feature_matrix.append(processed_feature_data)
        mask_feature_data = create_mask(processed_feature_data)  # 将空的mask
        mask_history_feature_matrix.append(mask_feature_data)
        
    # print('mask',len(origin_history_feature_matrix),len(origin_history_feature_matrix[0]))
    return origin_history_feature_matrix, mask_history_feature_matrix
# 将记录mask
# def history_feature_mask(user_history_feature_index, data_matrix_user_history, max_history_len):
#     mask_history_feature_matrix = []
#     origin_history_feature_matrix = []
#     for feature_index in range(len(user_history_feature_index)):
#         feature_data = [data_row[feature_index] for data_row in data_matrix_user_history]  # 获取一列特征值
#         origin_history_feature_matrix.append(feature_data)
#         mask_feature_data = create_mask(feature_data)  # 将空的mask
#         mask_history_feature_matrix.append(mask_feature_data)
#         
#     # print('mask',len(origin_history_feature_matrix),len(origin_history_feature_matrix[0]))
#     return origin_history_feature_matrix, mask_history_feature_matrix

# 将输入形成的data_hash和连续、离散特征列名,按照划分的训练或测试的user_id的列表，提取用户特征形成张量矩阵存储到data_tensor_hash中，以user_id为key，多个张量矩阵为value
def get_feature_to_matrix(train_or_val_or_test_list, data_hash, feature_column_dict):
    # 存储新的张量hash
    data_tensor_hash = {}
    # 存储历史记录的掩码矩阵
    data_tensor_history_mask_hash = {}
    target_label = []  # 预测目标值的标签

    for user_id in train_or_val_or_test_list:
        user_data = data_hash[user_id]
        # 创建空的二维矩阵
        data_matrix_history_continue = []
        data_matrix_history_discrete = []
        data_matrix_target_continue = []
        data_matrix_target_discrete = []
#         target_label = []
        # 提取特征列对应的索引
        # user_feature_continue_index = [user_data.columns.get_loc(col) for col in feature_column_dict['user_info_continue'] if col in user_data.columns]
        # user_feature_discrete_index = [user_data.columns.get_loc(col) for col in feature_column_dict['user_info_discrete'] if
        #                                col in user_data.columns]
        total_continue_index = [user_data.columns.get_loc(col) for col in feature_column_dict['total_continue_feature'] if col in user_data.columns]
        total_discrete_index = [user_data.columns.get_loc(col) for col in feature_column_dict['total_discrete_feature'] if col in user_data.columns]
        D_index = [user_data.columns.get_loc(col) for col in feature_column_dict['D'] if col in user_data.columns]
                
        # 填充数据矩阵
        for i in range(len(user_data)):
            data_matrix_history_continue.append(
                [user_data.iloc[i, col] for col in total_continue_index])  # 用户历史连续特征
            data_matrix_history_discrete.append(
                [user_data.iloc[i, col] for col in total_discrete_index])  # 用户历史离散特征
            target_label.append(user_data.iloc[i, -1])  # 预测目标的y值

        # 将numpy数组转换为PyTorch张量       
        # history   得到的data_matrix_user_history及data_tensor_pay_QOE_continue维度是(feature_num,history_len)需要转成tensor后转置

        data_tensor_history_continue = torch.tensor(np.array(data_matrix_history_continue), dtype=torch.float32)
        data_tensor_history_discrete = torch.tensor(np.array(data_matrix_history_discrete), dtype=torch.float32)
        data_label_tensor = torch.tensor(np.array(target_label), dtype=torch.float32)
#         print('data_label_tensor',data_label_tensor.shape)

#         data_tensor_history_continue = torch.transpose(torch.tensor(np.array(data_matrix_history_continue), dtype=torch.float32),0,1)
#         data_tensor_history_discrete = torch.transpose(torch.tensor(np.array(data_matrix_history_discrete), dtype=torch.float32),0,1)
#         data_tensor_history_continue_mask = []
#         data_tensor_history_discrete_mask = []
#         print('data_tensor_history_discrete',data_tensor_history_discrete.shape)
        
        # user + target   输出维度为（1，feature_num）,一处第一个为1的维度变为（feature_num）
#         data_tensor_target_continue = torch.squeeze(torch.tensor(np.array(data_matrix_target_continue), dtype=torch.float32),dim=0)
#         data_tensor_target_discrete = torch.squeeze(torch.tensor(np.array(data_matrix_target_discrete), dtype=torch.float32),dim=0)
#         data_label_tensor = torch.squeeze(data_label_tensor,dim=0)
#         print('data_label_tensor',data_label_tensor.shape)
        
        # 生成hash值，按user_id为key存储成hash
        tensor_hash_value = {
            'history_continue': data_tensor_history_continue,
            'history_discrete': data_tensor_history_discrete,
#             'target_continue': data_tensor_target_continue,
#             'target_discrete': data_tensor_target_discrete,
            'label': data_label_tensor

        }
#         tensor_hash_value_history_mask = {
#             'history_continue': data_tensor_history_continue_mask,
#             'history_discrete': data_tensor_history_discrete_mask,
#         }
        if user_id in data_tensor_hash:
            data_tensor_hash[user_id].update(tensor_hash_value)
#             data_tensor_history_mask_hash[user_id].update(tensor_hash_value_history_mask)
        else:
            data_tensor_hash[user_id] = tensor_hash_value
#             data_tensor_history_mask_hash[user_id] = tensor_hash_value_history_mask
    
    # 如果需要合并成一个张量，可以使用torch.cat方法
    # combined_tensor = torch.cat((data_matrix_1_tensor, data_matrix_2_tensor), dim=1)
    # data_tensor_hash中用户历史的输出维度(max_history_len,feature_num)，目标的输出维度是（feature_num）
    return data_tensor_hash, target_label


# 张量矩阵添加一个batch维度，并在用户特征与目标特征的张量中再添加一维使其与用户历史行为张量对齐， 形成两种：
# 原数据为：1.用户特征与目标特征都为：（1,feature_num）; 2.用户历史行为特征为（max_history_len(固定长度的历史记录数),feature_num）
# 新数据为：1.用户特征与目标特征都为：（batch,1,1,feature_num); 2.用户历史行为特征为（batch,max_history_len(固定长度的历史记录数),feature_num）
# 形成batch维度的特征
def generate_batch_feature(train_or_val_or_test_list,data_tensor_hash, feature_category,is_label = False):  # 例:feature_category = 'user_info_continue' 就是上面生成的tensor_hash_value字典的键
    tensor_list = []
#     batch_feature_tensor = None
    for user_id in train_or_val_or_test_list:  # 遍历data_tensor_hash的所有key (user_id)
        if feature_category in data_tensor_hash[user_id]:
            tensor = data_tensor_hash[user_id][feature_category]  # 获取feature_category对应的张量
            tensor_list.append(tensor)  # 添加到tensor_list中
    if is_label:
        batch_feature_tensor = tensor_list[-1]
    else:
        batch_feature_tensor = torch.stack(tensor_list, dim=0)  # 在第一个维度上合并所有张量(其实相当于生成一个新维度)
    return batch_feature_tensor
# 生成batch再添加维度对齐张量（三个维度）
def generate_user_feature_alignment_tensor(train_or_val_or_test_list,data_tensor_hash,is_mask=False):
    # 用户历史行为矩阵（max_history_len(固定长度的历史记录数),feature_num）->（batch,max_history_len(固定长度的历史记录数),feature_num）
    history_continue_batch_feature_tensor = generate_batch_feature(train_or_val_or_test_list,data_tensor_hash, 'history_continue')
    history_discrete_batch_feature_tensor = generate_batch_feature(train_or_val_or_test_list,data_tensor_hash, 'history_discrete')
    table_tensor = generate_batch_feature(train_or_val_or_test_list,data_tensor_hash, 'label',is_label = True)
    # 看是否是掩码矩阵，不是则xxx，是则没有user+target
#     if is_mask==False:
#         # 用户矩阵 (feature_num) ->(batch,feature_num)
#         # user_info_continue_batch_feature_tensor = generate_batch_feature(data_tensor_hash, 'user_info_continue')
#         # user_info_discrete_batch_feature_tensor = generate_batch_feature(data_tensor_hash, 'user_info_discrete')
#         # 目标矩阵 (feature_num) ->(batch,feature_num)
#         target_continue_batch_feature_tensor = generate_batch_feature(train_or_val_or_test_list,data_tensor_hash, 'target_continue')
#         target_discrete_batch_feature_tensor = generate_batch_feature(train_or_val_or_test_list,data_tensor_hash, 'target_discrete')

#         # 假设原始张量矩阵为 tensor，形状为 (batch_size, feature_num)将其加一个维度变为 (batch_size, 1, feature_num)
#         # user_info_continue_batch_feature_tensor = torch.unsqueeze(user_info_continue_batch_feature_tensor, dim=1)
#         # user_info_discrete_batch_feature_tensor = torch.unsqueeze(user_info_discrete_batch_feature_tensor, dim=1)
#         target_continue_batch_feature_tensor = torch.unsqueeze(target_continue_batch_feature_tensor, dim=1)
#         target_discrete_batch_feature_tensor = torch.unsqueeze(target_discrete_batch_feature_tensor, dim=1)

#         batch_feature_tensor_dict = {
#             'history_discrete': history_discrete_batch_feature_tensor,
#             'history_continue': history_continue_batch_feature_tensor,
#             'target_discrete': target_discrete_batch_feature_tensor,     
#             'target_continue': target_continue_batch_feature_tensor,
#         }
#     else:
    batch_feature_tensor_dict = {
        'history_discrete': history_discrete_batch_feature_tensor,
        'history_continue': history_continue_batch_feature_tensor,
        'label':table_tensor
    }
    return batch_feature_tensor_dict  # 这里张量输出的全是三维 (batch_size, 1 or max_history_len, feature_num)


# 由于模型输入得是张量，因此在之前将字典转化为了张量，现在将它转换回去
class TensorDatasettoDict(Dataset):
    def __init__(self, dataset, keys):
        self.dataset = dataset
        self.keys = keys

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        data = self.dataset[idx]
        sample = {key: data[i] for i, key in enumerate(self.keys)}
        return sample

# test
# 获取训练、验证、测试集对应的数据形成的向量hash存储及label
# print(data_hash[3617476])
# train_data_tensor_hash, train_label, train_data_tensor_hash_history_mask = get_feature_to_matrix(train_list, data_hash, feature_column_dict)
# first_key = list(train_data_tensor_hash.keys())[0]
# print(train_data_tensor_hash[first_key]['pay_QOE_discrete'][:,0])
# print(train_label)
# # print(train_data_tensor_hash[3617476])
# dimensions1 = train_data_tensor_hash[3617476]['pay_QOE_continue'].size()
# dimensions2 = train_data_tensor_hash[3617476]['pay_QOE_discrete'].size()
# dimensions3 = train_data_tensor_hash[3617476]['pay_CHONGHE_continue'].size()
# dimensions4 = train_data_tensor_hash[3617476]['target_QOE_continue'].size()
# dimensions5 = train_data_tensor_hash[3617476]['target_QOE_discrete'].size()
# dimensions6 = train_data_tensor_hash[3617476]['target_CHONGHE_continue'].size()
# print("PyTorch张量的维度：", dimensions1,dimensions2,dimensions3,dimensions4,dimensions5,dimensions6)
# train_batch_feature_tensor_dict = generate_user_feature_alignment_tensor(train_list,train_data_tensor_hash)
# train_batch_feature_tensor_history_mask_dict = generate_user_feature_alignment_tensor(train_data_tensor_hash_history_mask,is_mask=True)
# print(train_batch_feature_tensor_dict['pay_QOE_discrete'][0,:,0])
# dimensions1 = train_data_tensor_hash[3617476]['pay_QOE_continue'].size()
# dimensions2 = train_data_tensor_hash[3617476]['pay_QOE_discrete'].size()
# dimensions3 = train_data_tensor_hash[3617476]['pay_CHONGHE_continue'].size()
# dimensions4 = train_data_tensor_hash[3617476]['target_QOE_continue'].size()
# dimensions5 = train_data_tensor_hash[3617476]['target_QOE_discrete'].size()
# dimensions6 = train_data_tensor_hash[3617476]['target_CHONGHE_continue'].size()
# print("PyTorch添加batch后张量的维度：", dimensions1,dimensions2,dimensions3,dimensions4,dimensions5,dimensions6)

In [11]:
# 3.基础模型 embedding、attention
# 构建离散特征的embedding
def discrete_embedding(feature_category_num_dict, feature_column_name_list, embedding_dim): # 输入特征取值大小的集合,特征数,维度
    # 创建一个列表来存储每个嵌入层
    embeddings = []
    for i in range(0, len(feature_column_name_list)):
        # print(feature_column_name_list[i], feature_category_num_dict[feature_column_name_list[i]])
        embedding_layer1 = nn.Embedding(feature_category_num_dict[feature_column_name_list[i]]+2, embedding_dim)
        embeddings.append(embedding_layer1)
    #     print('embedding维度',feature_category_num_dict[feature_column_name_list[i]]+1)
    # print('本轮embedding层：',len(feature_column_name_list))
    return embeddings

# 全连接层 MLP
def dense_layer(in_features, out_features):
    # in_features=hidden_size,out_features=1
    return nn.Sequential(
        nn.Linear(in_features, out_features, bias=True),
        nn.ReLU())
# 全连接层 MLP
def dense_layer_noReLu(in_features, out_features):
    # in_features=hidden_size,out_features=1
    return nn.Sequential(
        nn.Linear(in_features, out_features, bias=True))

# 连续特征离散化
def continuous_embedding(num_continuous_features, out_features):
    continuous_embedding_layers = []
    for i in range(0,len(num_continuous_features)):
        num_continuous_feature = num_continuous_features[i]
        embedding_layer = dense_layer(1, out_features)
        continuous_embedding_layers.append(embedding_layer)
    return continuous_embedding_layers

# 根据全特征数量表及类别，得到类别下的对应特征数量  feature_column_name_list = feature_column_dict['user_info_continue']
def category_feature_num(feature_category_num_dict, feature_column_name_list):
    category_feature_num_list = []
    for i in range(len(feature_column_name_list)):
        category_feature_num_list.append(feature_category_num_dict[feature_column_name_list[i]])
    # print('category_feature_num',len(category_feature_num_list))
    return category_feature_num_list 

# SE层中找到合适的reduction使channel // reduction得到整数
def find_reduction(channel, min_reduction=2, max_reduction=19):  
    # 对于质数，直接取自己作为reduction  
    if is_prime(channel):  
        return channel  
      
    # 计算介于min_reduction和max_reduction之间的候选reduction值  
    candidates = [i for i in range(min_reduction, max_reduction + 1) if channel % i == 0]  
      
    # 如果候选列表为空，则至少取2作为reduction  
    if not candidates:  
        return min_reduction  
      
    # 尝试找到最大的候选值，使得channel // reduction的结果尽可能大  
    reduction = max(candidates)  
      
    return reduction  
def is_prime(n):  
    """判断一个数是否为质数"""  
    if n < 2:  
        return False  
    for i in range(2, int(math.sqrt(n)) + 1):  
        if n % i == 0:  
            return False  
    return True  
    
# 输入(batch,feature_num,embedding_dim,1) ->(batch,feature_num,embedding_dim,1)->输出特征权重及权重乘后的(batch,embedding_dim) 
class SELayer(nn.Module):
    def __init__(self, channel, reduction=16):
        super(SELayer, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.reduction = reduction
        self.reduction = find_reduction(channel)
        self.fc = nn.Sequential(       
            nn.Linear(channel, channel // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channel // reduction, channel, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, h, w = x.size()
        # print('b, c, h, w',b, c, h, w)
        y = self.avg_pool(x).view(b, c)   
        # print('y',y)
        weight = self.fc(y).view(b, c, 1, 1)
        new_x = x * weight.expand_as(x)  # 利用了 PyTorch 的广播机制，使得张量 weight 被复制成与输入 x 相同的形状，然后进行逐元素相乘 
        # 加权平均 (batch, embedding_dim)
        weighted_avg_out_x = new_x.mean(dim=1, keepdim=True)  # 在 feature_num维度上取平均，保持维度
        # 调整维度
        weighted_avg_out_x = weighted_avg_out_x.view(b, 1, h, w)
        # 去除最后一维
        new_x = new_x.squeeze(dim=3)
        weighted_avg_out_x = weighted_avg_out_x.squeeze(dim=3)
        
        return  weight, weighted_avg_out_x,new_x
   
    # 多头自注意力
class MultiHeadSelfAttention(nn.Module):
    def __init__(self, num_heads, feature_dim, max_history_len):
        super(MultiHeadSelfAttention, self).__init__()
        self.num_heads = num_heads  #10
        self.feature_dim = feature_dim   #200
        self.head_dim = feature_dim // num_heads
        self.max_history_len = max_history_len

        self.WQ = nn.Linear(feature_dim, feature_dim)
        self.WK = nn.Linear(feature_dim, feature_dim)
        self.WV = nn.Linear(feature_dim, feature_dim)

    def forward(self, history_matrix, mask=None):
        batch_size, history_len, _ = history_matrix.size()

        Q = self.WQ(history_matrix)
        K = self.WK(history_matrix)
        V = self.WV(history_matrix)

        Q = Q.view(batch_size, history_len, self.num_heads, self.head_dim).permute(0, 2, 1, 3)  #(batch,num_heads,history_len,head_dim)
        K = K.view(batch_size, history_len, self.num_heads, self.head_dim).permute(0, 2, 1, 3)
        V = V.view(batch_size, history_len, self.num_heads, self.head_dim).permute(0, 2, 1, 3)

        attention_scores = torch.matmul(Q, K.permute(0, 1, 3, 2)) / (self.head_dim ** 0.5)

        if mask is not None:
            mask = mask.permute(0, 2, 1)  # 二、三维度互换  变为(batch, feature_num, history)
            temp_dim=mask.shape[1]
            #（样本数*特征数,历史数）
            mask=mask.reshape(-1,max_history_len)
            attention_scores = attention_scores.masked_fill(mask.unsqueeze(1).unsqueeze(2).bool(), float('-inf'))  #()

        attention_weights = torch.softmax(attention_scores, dim=-1)  #shape(batch,head,history_len,history_len)
        #(batch,history_len,200)
        weighted_sum = torch.matmul(attention_weights, V).permute(0, 2, 1, 3).contiguous().view(batch_size, history_len,
                                                                                                self.feature_dim)
        # 计算加权平均
        weighted_avg_out = weighted_sum.mean(dim=1, keepdim=True)  # 在 history_len 维度上取平均，保持维度
        # 调整维度
        weighted_avg_out = weighted_avg_out.view(batch_size, 1,self.feature_dim)
        # print('weighted_sum',weighted_avg_out.shape)
        
        return attention_weights, weighted_avg_out, weighted_sum

# 注意力机制 关于用
class MultiHeadHistory_TargetAttention(nn.Module):
    def __init__(self, num_heads, embed_dim, dropout=0.1):
        super(MultiHeadHistory_TargetAttention, self).__init__()
        
        assert embed_dim % num_heads == 0, f"Embedding dimension ({embed_dim}) should be divisible by the number of heads ({num_heads})."

        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        
        # 定义权重矩阵
        self.q_linear = nn.Linear(embed_dim, embed_dim)
        self.k_linear = nn.Linear(embed_dim, embed_dim)
        self.v_linear = nn.Linear(embed_dim, embed_dim)
        
        self.out_proj = nn.Linear(embed_dim, embed_dim)

        self.scaling = self.head_dim ** -0.5
        self.dropout = nn.Dropout(dropout)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, query, key, value, attn_mask=None):
        batch_size = query.size(0)       
        # 进行线性投影并分离成多个头
        q = self.q_linear(query).view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)
        k = self.k_linear(key).view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)
        v = self.v_linear(value).view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)
        # 计算注意力得分
        scores = torch.matmul(q, k.transpose(-2, -1)) * self.scaling
        if attn_mask is not None:
            scores.masked_fill_(attn_mask.unsqueeze(1), float('-inf'))
        # 应用softmax函数
        attn_weights = self.softmax(scores)
        # 应用dropout
        attn_weights = self.dropout(attn_weights)
        # 进行值的加权求和
        context = torch.matmul(attn_weights, v).transpose(1, 2).contiguous().view(batch_size, -1, self.embed_dim)
        # 输出层的线性变换
        output = self.out_proj(context)
        return attn_weights, output

# BiLSTM
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class BiLSTMWithBatchNorm(nn.Module):  
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers):  
        super(BiLSTMWithBatchNorm, self).__init__()  
          
        self.hidden_dim = hidden_dim  
        self.num_layers = num_layers  
          
        # 定义BiLSTM层，使用nn.ModuleList来容纳多个层  
        self.lstm_layers = nn.ModuleList([  
            nn.LSTM(input_dim if i == 0 else hidden_dim * 2, hidden_dim, 1, batch_first=True, bidirectional=True)  
            for i in range(num_layers)  
        ])  
          
        # 定义BatchNorm层，与LSTM层数量相同  
        self.batch_norm_layers = nn.ModuleList([  
            nn.BatchNorm1d(hidden_dim * 2)  
            for _ in range(num_layers)  
        ])  
          
        # 全连接层  
        self.fc = nn.Linear(hidden_dim * 2, output_dim)  
        
    def forward(self, x):  
        # 初始化所有层的隐藏状态和细胞状态  
        # 注意这里使用列表来存储每一层的隐藏状态和细胞状态  
        h0 = [torch.zeros(2, x.size(0), self.hidden_dim).to(x.device) for _ in range(self.num_layers)]  
        c0 = [torch.zeros(2, x.size(0), self.hidden_dim).to(x.device) for _ in range(self.num_layers)]  

        # 遍历每一层LSTM  
        for i, (lstm_layer, batch_norm_layer) in enumerate(zip(self.lstm_layers, self.batch_norm_layers)):  
            # LSTM前向传播  
            # 注意这里使用h0[i]和c0[i]作为当前层的初始隐藏状态和细胞状态  
            out, (hn, cn) = lstm_layer(x, (h0[i], c0[i]))  
            # 应用批量标准化  
            out = batch_norm_layer(out.transpose(1, 2)).transpose(1, 2)  
            # 更新x为当前层的输出，用于下一层的输入  
            x = out  
            # 更新当前层的隐藏状态和细胞状态，用于下一层  
            h0[i] = hn  
            c0[i] = cn  

        # 取最后一个时间步的输出，注意此时out已经是最后一层的输出  
        out = out[:, -1, :]  
        # 全连接层  
        out = self.fc(out)  

        return out     
#     def forward(self, x):  
#         # 初始化隐藏状态和细胞状态  
#         h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim * 2).to(x.device)  
#         c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim * 2).to(x.device)  
          
#         # 遍历每一层LSTM  
#         for lstm_layer, batch_norm_layer in zip(self.lstm_layers, self.batch_norm_layers):  
#             # LSTM前向传播  
#             out, (hn, cn) = lstm_layer(x, (h0, c0))  
#             # 应用批量标准化  
#             out = batch_norm_layer(out.transpose(1, 2)).transpose(1, 2)  
#             # 更新隐藏状态和细胞状态为下一层做准备  
#             h0 = hn  
#             c0 = cn  
#             # 使用当前层的输出作为下一层的输入  
#             x = out   
#         # 取最后一个时间步的输出  
#         out = out[:, -1, :]  
#         # 全连接层  
#         out = self.fc(out)  
          
#         return out  


# class BiLSTMBN(nn.Module):
#     def __init__(self, input_size, hidden_size=64, num_layers=4, dropout=0.2, output_dim=1):
#         super(BiLSTMBN, self).__init__()
        
#         # 定义每层双向LSTM和对应的批量归一化层
#         self.lstm_layers = nn.ModuleList()
#         self.bn_layers = nn.ModuleList()
#         for _ in range(num_layers):
#             lstm_layer = nn.LSTM(input_size if layer_num == 0 else hidden_size * 2, hidden_size,
#                                 bidirectional=True, batch_first=True, dropout=dropout)
#             bn_layer = nn.BatchNorm1d(hidden_size * 2)
#             self.lstm_layers.append(lstm_layer)
#             self.bn_layers.append(bn_layer)
#             input_size = hidden_size * 2  # 下一层的输入维度是上一层输出的维度

#         # 输出层，假设我们进行分类任务，类别数为output_dim
#         self.fc = nn.Linear(hidden_size * 2, output_dim)  # 注意：因为是双向所以hidden_size要乘以2
        
#         # 初始化权重
#         self.init_weights()
        
#     def init_weights(self):
#         for name, param in self.named_parameters():
#             if 'weight_ih' in name:
#                 nn.init.xavier_uniform_(param.data)
#             elif 'weight_hh' in name:
#                 nn.init.orthogonal_(param.data)
#             elif 'bias' in name:
#                 param.data.fill_(0)

#     def forward(self, inputs, lengths=None):
#         packed_input = nn.utils.rnn.pack_padded_sequence(inputs, lengths, batch_first=True, enforce_sorted=False)

#         hidden_states = []
#         for i, (lstm_layer, bn_layer) in enumerate(zip(self.lstm_layers, self.bn_layers)):
#             # 通过当前LSTM层
#             lstm_outs, (hidden, cell) = lstm_layer(packed_input)
            
#             # 对隐藏状态应用批量归一化
#             lstm_outs_unpacked, _ = nn.utils.rnn.pad_packed_sequence(lstm_outs, batch_first=True)
#             hidden_forward = lstm_outs_unpacked[:, :, :hidden_size]
#             hidden_backward = lstm_outs_unpacked[:, :, hidden_size:]
#             normalized_hidden = torch.cat([bn_layer(hidden_forward), bn_layer(hidden_backward)], dim=-1)

#             # 将处理后的隐藏状态存储起来，并作为下一层的输入
#             hidden_states.append(normalized_hidden)
#             packed_input = nn.utils.rnn.pack_padded_sequence(normalized_hidden, lengths, batch_first=True, enforce_sorted=False)

#         # 取最后一层的隐藏状态
#         concat_hidden = hidden_states[-1]

#         # 经过全连接层进行分类
#         out = self.fc(concat_hidden)

#         return out

# class BiLSTM(nn.Module):
#     def __init__(self, input_size, hidden_size=64, num_layers=1, dropout=0.2):
#         super(BiLSTM, self).__init__()
        
#         # 定义双向LSTM层
#         self.lstm = nn.LSTM(input_size, hidden_size, num_layers=num_layers, bidirectional=True, batch_first=True, dropout=dropout)
        
#         # 输出层，假设我们进行分类任务，类别数为output_dim
#         self.fc = nn.Linear(hidden_size * 2, output_dim)  # 注意：因为是双向所以hidden_size要乘以2
        
#         # 初始化权重
#         self.init_weights()

#     def init_weights(self):
#         for name, param in self.named_parameters():
#             if 'weight_ih' in name:
#                 nn.init.xavier_uniform_(param.data)
#             elif 'weight_hh' in name:
#                 nn.init.orthogonal_(param.data)
#             elif 'bias' in name:
#                 param.data.fill_(0)

#     def forward(self, inputs, lengths=None):
#         # 对序列数据进行排序（按长度降序），然后反向恢复原顺序
#         packed_input = nn.utils.rnn.pack_padded_sequence(inputs, lengths, batch_first=True, enforce_sorted=False)
        
#         # 通过双向LSTM层
#         lstm_outs, (hidden, cell) = self.lstm(packed_input)
        
#         # 对输出结果进行解压，并取最后一个时间步的隐藏状态作为最终特征表示
#         lstm_outs, _ = nn.utils.rnn.pad_packed_sequence(lstm_outs, batch_first=True)
#         last_time_step_output = lstm_outs[:, -1, :]
        
#         # 将双方向的隐藏状态拼接在一起，然后通过全连接层进行分类
#         concat_hidden = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)
#         out = self.fc(concat_hidden)
        
#         return out



In [12]:
# 4.Embedding层

# user_history_feature 对于一个user的多个历史行为，将其拼接成一维向量 要先经过一层通道注意力机制得到最后结果
# (样本数,history,20,200) ->多头 ->(样本数,20,200)->转置->(样本数,200,20) ->SE->特征权重->(样本数,200,20) ->转置-> 加权->(样本数,1，200)
# user_pay_history_feature 加上batch的
# 用户历史
class UserPayHistoryEmbedding(nn.Module):
    def __init__(self, continue_embedding_dim, discrete_embedding_dim, feature_category_num_dict, feature_column_dict):
        super(UserPayHistoryEmbedding, self).__init__()
        # 连续特征
        # 离散特征
        self.feature_category_num_dict = feature_category_num_dict
        # 离散embedding
        self.history_discrete_embeddings = discrete_embedding(self.feature_category_num_dict,
                                                                feature_column_dict['total_discrete_feature'],
                                                                discrete_embedding_dim)
        # MLP  连续embedding
        self.history_continue_embedding = continuous_embedding(category_feature_num(feature_category_num_dict, feature_column_dict['total_continue_feature']), continue_embedding_dim)
       
    def forward(self, batch_feature_tensor_history_discrete,batch_feature_tensor_history_continue):
        # user_history Embedding
        # user_history_continue_features_embedding 得到(batch, 1, continue_feature_num, continue_embedding_dim)
        # user_history_discrete_features_embedding 得到(batch, 1, discrete_feature_num, discrete_embedding_dim)
        # history中有三种：QOE/CHONGHE/FUFEI,将其分别转化为embedding然后合并
        # embedding的数据要求输入是整数类型 因此转为int，输入数据得是从0开始的索引后的数据，因此mask后得到-1以及在输入时得到了从0开始的索引后值，
        # 现在所有discrete数据输入时+1，即 batch_feature_tensor_pay_QOE_discrete[:, :, i]+1 
        # for i in range(batch_feature_tensor_pay_QOE_discrete.shape[2]):
        #     print(i,batch_feature_tensor_pay_QOE_discrete.shape[2],batch_feature_tensor_pay_QOE_discrete[:, :, i]+1,self.user_pay_history_QOE_discrete_embeddings[i].num_embeddings )
        batch_feature_tensor_history_discrete = batch_feature_tensor_history_discrete.int()
        history_discrete_column_discrete_features_embedding = torch.stack(
            [embedding_layer(batch_feature_tensor_history_discrete[:, :, i]+1) for i, embedding_layer in
             enumerate(self.history_discrete_embeddings)], dim=-2)
        history_continue_column_discrete_features_embedding = torch.stack(
            [embedding_layer(batch_feature_tensor_history_continue[:,:, i].unsqueeze(-1).float()) for i, embedding_layer in
             enumerate(self.history_continue_embedding)], dim=-2)
        history_vec = torch.cat(
            [history_discrete_column_discrete_features_embedding, history_continue_column_discrete_features_embedding], dim=2)  # 特征级合并
        
        return history_vec


In [13]:
# 6.整合模型

# (batch,600)经过网络变成200 +(batch,featuer_user*200)经过网络变成200 -> (batch,200)
# (batch,200) ->MLP ->(batch，1) ->sigmoid -> (batch,1)

# 整合层
class MatchingModel(nn.Module):
    def __init__(self, feature_category_num_dict, feature_column_dict, continue_embedding_dim,
                 discrete_embedding_dim,num_heads, feature_dim, max_history_len):
        super(MatchingModel, self).__init__()
        # Embedding层
        # self.user_info_embedding_layer = UserInfoEmbedding(continue_embedding_dim, discrete_embedding_dim, feature_category_num_dict, feature_column_dict)
#         self.history_embedding_layer = UserPayHistoryEmbedding(continue_embedding_dim, discrete_embedding_dim, feature_category_num_dict, feature_column_dict)
        #print('embedding user_history结果')
#         self.target_embedding_layer = TargetEmbedding(continue_embedding_dim, discrete_embedding_dim, feature_category_num_dict, feature_column_dict)
        # BiLSTM层  每层64个神经元 输出进入批量归一化 共4层
        feature_num = len(feature_column_dict['total_discrete_feature'])+len(feature_column_dict['total_continue_feature'])
        self.bilstm_layer = BiLSTMWithBatchNorm(input_dim=feature_num, hidden_dim=64, output_dim=1, num_layers=4)

        # MLP
        final_dim =200
        self.pay_vec_MLP_layer = dense_layer_noReLu(final_dim, 1)


    def forward(self, batch_feature_tensor_history_discrete,batch_feature_tensor_history_continue,
                label_tensor):
        # Embedding层
#         history_vec= self.history_embedding_layer(batch_feature_tensor_history_discrete,batch_feature_tensor_history_continue)        
#         target_vec = self.target_embedding_layer(batch_feature_tensor_target_discrete,batch_feature_tensor_target_continue)
        history_vec = torch.cat([batch_feature_tensor_history_discrete, batch_feature_tensor_history_continue], dim=2)  
#         print('history_vec',history_vec.shape)
        # BiLSTM层  (batch,1)
        out_vec = self.bilstm_layer(history_vec)
#         print('lstm_out',out_vec.shape)
        
        # 使用softmax函数将logits转换为概率分布
        sigmoid_score = torch.sigmoid(out_vec)  # 在类别维度（dim=1）上应用softmax
        softmax_score = torch.softmax(out_vec, dim=1)

        return softmax_score, sigmoid_score

# 损失函数
class LossFunction(nn.Module):
    def __init__(self):
        super(LossFunction, self).__init__()

    def forward(self, pred, target_label):
        # pred是未经处理过的原值，target_label是0、1标签
        # 计算第一个任务的二元交叉熵损失
        loss = F.binary_cross_entropy_with_logits(pred, target_label, reduction='none')
        return loss


# 自动评估阈值，计算ACC 、 Precision 等评估指标
def evaluate(y_true, y_pred, digits=4, cutoff='auto'):
    '''
    Args:
        y_true: list, labels, y_pred: list, predictions, digits: The number of decimals to use when rounding the number. Default is 4（保留小数后几位）
        cutoff: float or 'auto'
    Returns:
        evaluation: dict
    '''
    # 根据预测概率值y_pred计算最佳的切分阈值
    if cutoff == 'auto':
        fpr, tpr, thresholds = roc_curve(y_true, y_pred)
        youden = tpr-fpr
        cutoff = thresholds[np.argmax(youden)]
    y_pred_t = [1 if i > cutoff else 0 for i in y_pred]

    evaluation = OrderedDict()
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred_t).ravel()
    evaluation['auc'] = round(roc_auc_score(y_true, y_pred), digits)
    evaluation['acc'] = round(accuracy_score(y_true, y_pred_t), digits)
    evaluation['recall'] = round(recall_score(y_true, y_pred_t), digits)
    evaluation['precision'] = round(precision_score(y_true, y_pred_t), digits)
    evaluation['specificity'] = round(tn / (tn + fp), digits)
    evaluation['F1'] = round(f1_score(y_true, y_pred_t), digits)
    evaluation['cutoff'] = cutoff

    return evaluation


In [14]:
# 创建大模型的实例 'drama_upuser_subscriptions_num,drama_sound_max_traffic_position_in_sound_avg,label1'
# model = MatchingModel(feature_category_num_dict, feature_column_dict, continue_embedding_dim,
#                  discrete_embedding_dim, num_heads, feature_dim, max_history_len)
# print('模型创建完成')

In [15]:
# 7.模型训练 Trainging

def model_training(model,train_loader,val_loader, lossfunction,optimizer,EPOCH,device):
    # 定义早停策略的参数
    best_val_loss = float('inf')  # 初始化最佳验证损失为正无穷
    patience = 1  # 容忍多少个epoch没有验证性能提升
    early_stopping_counter = 0  # 初始化计数器
    
    for epoch in range(EPOCH):
        model.train()  # 设置模型为训练模式
        total_classfier_loss = 0.0
        total_loss = 0.0
        train_time = 0
        val_time = 0
        for batch in train_loader:
            batch = [data.to(device) for data in batch]
            batch_feature_tensor_history_discrete,batch_feature_tensor_history_continue,train_label_tensor = batch  
            for param in model.parameters():
                param.requires_grad = True
            optimizer.zero_grad()
            softmax_score, sigmoid_score = model(batch_feature_tensor_history_discrete,batch_feature_tensor_history_continue,
                                                train_label_tensor)  

            # sigmoid
            # print('sigmoid_score',sigmoid_score)
            sigmoid_score = sigmoid_score[:, 0]  # (样本数，1)
            train_label_tensor = train_label_tensor[:, 0].to(device)  # (样本数，1)
            train_label_tensor[train_label_tensor == 1] = 0
            train_label_tensor[train_label_tensor == 2] = 1
            # train_label_tensor = torch.where(train_label_tensor == 1, torch.tensor(0).to(device), torch.tensor(1).to(device))  # 使用 torch.where 将 1 映射为 0，将 2 映射为 1
            loss = lossfunction(sigmoid_score, train_label_tensor.float())
            # softmax
            # softmax_score = softmax_score[:, 0]  # (样本数，1)
            # train_label_tensor = train_label_tensor[:, 0].to(device)  # (样本数，1)
            # train_label_tensor = torch.where(train_label_tensor == 1, torch.tensor(0).to(device), torch.tensor(1).to(device))  # 使用 torch.where 将 1 映射为 0，将 2 映射为 1
            # loss = lossfunction(softmax_score, train_label_tensor.float())
            loss.to(device)

            
             # loss回传检查
            # for name, parms in model.named_parameters():	
            #     if parms.grad is not None:  # 检查梯度是否为None
            #         grad_mean = torch.mean(parms.grad)  # 计算梯度的均值
            #         print('-->name:', name, '-->grad_requirs:', parms.requires_grad, '-->grad_mean: {:.4f}'.format(grad_mean))
            #     else:
            #         print('-->name:', name, '-->grad_requirs:', parms.requires_grad, '-->grad_mean: None')
            loss.backward()
            optimizer.step()
            # print("=============更新之后===========")
            # for name, parms in model.named_parameters():	
            #     if parms.grad is not None:  # 检查梯度是否为None
            #         grad_mean = torch.mean(parms.grad)  # 计算梯度的均值
            #         print('-->name:', name, '-->grad_requirs:', parms.requires_grad, '-->grad_mean: {:.4f}'.format(grad_mean))
            #     else:
            #         print('-->name:', name, '-->grad_requirs:', parms.requires_grad, '-->grad_mean: None')
            # print(optimizer)
            # input("=====迭代结束=====")

            # 损失
            total_loss += loss.item()
            train_time += 1
            print('||--训练：----------',train_time,'个batch运行时间：',datetime.datetime.now(),'-------------')
        # 平均损失
        average_loss = total_loss / len(train_loader)
        

        if (epoch + 1) % 5 == 0:
            print(
                f"Epoch {epoch + 1},loss:{average_loss}")

            # 验证集评估
            model.eval()  # 将模型切换为评估模式
            with torch.no_grad():  # 在评估模式下不计算梯度
                total_loss_val = 0.0
                total_auc_val = 0.0
                total_acc_val = 0
                total_f1_val = 0
                total_precision_val = 0
                total_recall_val = 0
                val_time = 0
                for batch_val in val_loader:  # 假设你有一个名为 val_loader 的验证集数据加载器
                    batch_val = [data.to(device) for data in batch_val]
                    val_batch_feature_tensor_history_discrete,val_batch_feature_tensor_history_continue,val_label_tensor = batch_val 
                    softmax_score_val, sigmoid_score_val = model(val_batch_feature_tensor_history_discrete,val_batch_feature_tensor_history_continue,
                                                                        val_label_tensor) 

                    # sigmoid                   
                    sigmoid_score_val = sigmoid_score_val[:, 0]  # (样本数，1)
                    sigmoid_score_val = sigmoid_score_val.cpu()# .detach()  # 转为CPU
                    val_label_tensor = val_label_tensor[:, 0]  # (样本数，1)
                    val_label_tensor = val_label_tensor.cpu()
                    val_label_tensor[val_label_tensor == 1] = 0
                    val_label_tensor[val_label_tensor == 2] = 1
                    # val_label_tensor = torch.where(val_label_tensor == 1, torch.tensor(0), torch.tensor(1))  # 使用 torch.where 将 1 映射为 0，将 2 映射为 1
                    loss_val = lossfunction(sigmoid_score_val, val_label_tensor.float())
                    # softmax
                    # softmax_score_val = softmax_score_val[:, 0]  # (样本数，1)
                    # softmax_score_val = softmax_score_val.cpu()# .detach()  # 转为CPU
                    # val_label_tensor = val_label_tensor[:, 0]  # (样本数，1)
                    # val_label_tensor = val_label_tensor.cpu()
                    # val_label_tensor = torch.where(val_label_tensor == 1, torch.tensor(0), torch.tensor(1))  # 使用 torch.where 将 1 映射为 0，将 2 映射为 1
                    # loss_val = lossfunction(softmax_score_val, val_label_tensor.float())
    
                    # 损失
                    total_loss_val += loss_val.item()
                    # 计算验证集上的精度
                    # predicted_classes_val = (sigmoid_score_val > 0.5).long()
                    # total_acc_val += (predicted_classes_val == val_label_tensor).sum().item() / len(val_label_tensor)
                    # total_f1_val += f1_score(val_label_tensor, predicted_classes_val)
                    # total_recall_val += recall_score(val_label_tensor, predicted_classes_val)
                    # precision_val = ((predicted_classes_val == 1) & (val_label_tensor == 1)).sum().item() / (predicted_classes_val == 1).sum().item()
                    # total_precision_val += precision_val
                    # total_auc_val += roc_auc_score(val_label_tensor, softmax_score_val)
                    evaluation = evaluate(val_label_tensor,sigmoid_score_val)
                    total_acc_val += evaluation['acc']
                    total_f1_val += evaluation['F1']
                    total_recall_val += evaluation['recall']
                    total_precision_val += evaluation['precision']
                    total_auc_val += evaluation['auc']
                    
                    val_time += 1
                    print('||--验证：----------',val_time,'个batch运行时间：',datetime.datetime.now(),'-------------')
                # 平均损失
                average_loss_val = total_loss_val / len(val_loader)
                average_auc_val = total_auc_val / len(val_loader)
                average_acc_val = total_acc_val / len(val_loader)
                average_f1_val = total_f1_val / len(val_loader)
                average_precision_val = total_precision_val / len(val_loader)
                average_recall_val = total_recall_val / len(val_loader)
                print(f"Validation Loss: {average_loss_val},AUC: {average_auc_val},ACC:{average_acc_val},F1:{average_f1_val},Precision:{average_precision_val},Recall:{average_recall_val}") 

                if average_loss_val < best_val_loss:
                    best_val_loss = average_loss_val
                    early_stopping_counter = 0
                else:
                    early_stopping_counter += 1
                if early_stopping_counter >= patience:
                    print(f"早停策略触发，停止训练在第 {epoch} 个epoch.")
                    break

In [16]:
# 模型测试 Test

def test_model(model, test_loader):
    model.eval()  # 设置模型为评估模式
    with torch.no_grad():  # 在评估模式下不计算梯度
        total_loss_test = 0.0
        total_auc_test = 0.0
        total_acc_test  = 0
        total_f1_test = 0
        total_precision_test = 0
        total_recall_test = 0
        test_time = 0
        results = []  # 用于保存结果的列表
        for batch_test in test_loader:  # 假设你有一个名为 val_loader 的验证集数据加载器
            batch_test = [data.to(device) for data in batch_test]
            test_batch_feature_tensor_history_discrete,test_batch_feature_tensor_history_continue,test_label_tensor = batch_test 
            softmax_score_test, sigmoid_score_test = model(test_batch_feature_tensor_history_discrete,test_batch_feature_tensor_history_continue,
                                                                test_label_tensor)  
            # sigmoid
            sigmoid_score_test = sigmoid_score_test[:, 0]  # (样本数，1)
            sigmoid_score_test = sigmoid_score_test.cpu()#.detach()  # 转为CPU
            test_label_tensor = test_label_tensor[:, 0]  # (样本数，1)
            test_label_tensor = test_label_tensor.cpu()
            test_label_tensor[test_label_tensor == 1] = 0
            test_label_tensor[test_label_tensor == 2] = 1
            # test_label_tensor = torch.where(test_label_tensor == 1, torch.tensor(0), torch.tensor(1))  # 使用 torch.where 将 1 映射为 0，将 2 映射为 1
            loss_test = lossfunction(sigmoid_score_test, test_label_tensor.float())
            # softmax
            # softmax_score_test = softmax_score_test[:, 0]  # (样本数，1)
            # softmax_score_test = softmax_score_test.cpu()#.detach()  # 转为CPU
            # test_label_tensor = test_label_tensor[:, 0]  # (样本数，1)
            # test_label_tensor = test_label_tensor.cpu()
            # test_label_tensor = torch.where(test_label_tensor == 1, torch.tensor(0), torch.tensor(1))  # 使用 torch.where 将 1 映射为 0，将 2 映射为 1
            # loss_test = lossfunction(softmax_score_test, test_label_tensor.float())
            
            # 损失
            total_loss_test += loss_test.item()
            # 计算验证集上的精度
            # predicted_classes_test = (sigmoid_score_test > 0.5).long()
            # total_acc_test += (predicted_classes_test == test_label_tensor).sum().item() / len(test_label_tensor)
            # total_f1_test += f1_score(test_label_tensor, predicted_classes_test)
            # total_recall_test += recall_score(test_label_tensor, predicted_classes_test)
            # precision_test = ((predicted_classes_test == 1) & (test_label_tensor == 1)).sum().item() / (predicted_classes_test == 1).sum().item()
            # total_precision_test += precision_test
            # total_auc_test += roc_auc_score(test_label_tensor, sigmoid_score_test)
            evaluation = evaluate(test_label_tensor,sigmoid_score_test)
            total_acc_test += evaluation['acc']
            total_f1_test += evaluation['F1']
            total_recall_test += evaluation['recall']
            total_precision_test += evaluation['precision']
            total_auc_test += evaluation['auc']

            test_time += 1
            print('||--测试：----------',test_time,'个batch运行时间：',datetime.datetime.now(),'-------------')
        # 平均损失
        average_loss_test = total_loss_test / len(test_loader)
        average_auc_test = total_auc_test / len(test_loader)
        average_acc_test = total_acc_test / len(test_loader)
        average_f1_test = total_f1_test / len(test_loader)
        average_precision_test = total_precision_test / len(test_loader)
        average_recall_test = total_recall_test / len(test_loader)
        print(
            f"Test Loss: {average_loss_test},AUC: {average_auc_test},ACC:{average_acc_test},F1:{average_f1_test},Precision:{average_precision_test},Recall:{average_recall_test}")
        return average_loss_test, average_auc_test,average_acc_test,average_f1_test,average_precision_test,average_recall_test


In [17]:
# 模型运行
for data_time_windows in data_time_windows_list :
    
    path = './Dataset/' + data_time_windows + '_user_pay_pred_feature_deal.csv'
    dataset_spilt_path = './Dataset/' + data_time_windows + '_user_pay_pred_feature_spilt_BiLSTM.csv'
#     output_weight_result_path = './Dataset/' + data_time_windows + '_user_pay_pred_result_weight.csv'
    data_feature_continue_discrete_namelist_path = './Dataset/maoer_timewindows_continue_discrete_feature_column.csv'    # 连续与离散划分表
    # # 获取时间窗内连续与离散特征名的列表
    user_history_pay_QOE_continue_column, user_history_pay_CHONGHE_continue_column, \
    user_history_pay_FUFEI_continue_column, user_history_pay_QOE_discrete_column,\
    user_history_pay_CHONGHE_discrete_column, user_history_pay_FUFEI_discrete_column = get_continue_discrete_feature_namelist(data_time_windows, data_feature_continue_discrete_namelist_path)
    user_feature_continue_column = []
    user_feature_discrete_column = []

    # total continue feature
    total_continue_feature = user_history_pay_CHONGHE_continue_column+user_history_pay_FUFEI_continue_column
    total_discrete_feature = user_history_pay_CHONGHE_discrete_column+user_history_pay_FUFEI_discrete_column
    total_discrete_feature_add_D = user_feature_discrete_column+user_history_pay_QOE_discrete_column+user_history_pay_CHONGHE_discrete_column+user_history_pay_FUFEI_discrete_column
    total_discrete_feature_add_D.append('user_in_drama_is_pay_for_drama_in_next_time')
    D = 'user_in_drama_is_pay_for_drama_in_next_time'
    tensor_dict_idx = ['pay_QOE_continue','pay_QOE_discrete','pay_CHONGHE_continue','pay_CHONGHE_discrete','pay_FUFEI_continue','pay_FUFEI_discrete','target_QOE_continue','target_QOE_discrete','target_CHONGHE_continue','target_CHONGHE_discrete','target_FUFEI_continue','target_FUFEI_discrete']
    # print(len(user_history_pay_QOE_continue_column),len(user_history_pay_CHONGHE_continue_column),len(user_history_pay_FUFEI_continue_column))
    # 形成对应需要的特征名称列表
    feature_column_dict = {
        'total_continue_feature': total_continue_feature,
        'total_discrete_feature': total_discrete_feature,
        'D':D
    }
    # 创建一个空的DataFrame来存储结果
    test_auc_df = pd.DataFrame(columns=['时间','model','运行位置','Type','dataset','train_ratio','feature_embedding','batchSize','lr','max_history_len','实验数', '测试集总损失', 'AUC','ACC','F1','Precision','Recall'])
    test_weight_df = pd.DataFrame(columns=['时间','model','运行位置','Type','dataset','train_ratio','feature_embedding','batchSize','lr','max_history_len','实验数','se_user_pay_QOE_weight','se_user_pay_CHONGHE_weight','se_user_pay_FUFEI_weight','se_target_QOE_weight','se_target_CHONGHE_weight','se_target_FUFEI_weight',\
                                        'target_history_pay_attention_QOE_weight','target_history_pay_attention_CHONGHE_weight','target_history_pay_attention_FUFEI_weight'])
    for i in range(5):
        torch.autograd.set_detect_anomaly(True)
        print(f"i=:{i+1}")
        n = i
        # 数据集 train、val、test划分及总数据hash表(以user_id为key的存储对应对应行的hash表)及不同类特征数存储的字典
        train_list, val_list, test_list, data_hash, feature_category_num_dict = data_input(data_time_windows, path, dataset_spilt_path, train_ratio, val_ratio, test_ratio, total_continue_feature)
        # 获取训练、验证、测试集对应的数据形成的向量hash存储及label
        train_data_tensor_hash, train_label = get_feature_to_matrix(train_list, data_hash, feature_column_dict)
        val_data_tensor_hash, val_label = get_feature_to_matrix(val_list, data_hash, feature_column_dict)
        test_data_tensor_hash, test_label = get_feature_to_matrix(test_list, data_hash, feature_column_dict)
        # 输出查看结果
        # for key1 in train_data_tensor_hash.keys():
        #     dimensions1 = train_data_tensor_hash[key1]['pay_QOE_continue'].size()
        #     dimensions2 = train_data_tensor_hash[key1]['pay_QOE_discrete'].size()
        #     dimensions3 = train_data_tensor_hash[key1]['pay_CHONGHE_continue'].size()
        #     dimensions4 = train_data_tensor_hash[key1]['target_QOE_continue'].size()
        #     dimensions5 = train_data_tensor_hash[key1]['target_QOE_discrete'].size()
        #     dimensions6 = train_data_tensor_hash[key1]['target_CHONGHE_continue'].size()
        #     print("val_data_tensor_hash size=", dimensions1,dimensions2,dimensions3,dimensions4,dimensions5,dimensions6)

        # 生成batch再添加维度对齐张量（三个维度）这里张量输出的全是三维 (batch_size, 1 or max_history_len, feature_num)
        train_batch_feature_tensor_dict = generate_user_feature_alignment_tensor(train_list,train_data_tensor_hash)
        val_batch_feature_tensor_dict = generate_user_feature_alignment_tensor(val_list,val_data_tensor_hash)
        test_batch_feature_tensor_dict = generate_user_feature_alignment_tensor(test_list,test_data_tensor_hash)
        train_label_tensor = train_batch_feature_tensor_dict['label']
        val_label_tensor = val_batch_feature_tensor_dict['label']
        test_label_tensor = test_batch_feature_tensor_dict['label']
        train_label_tensor = train_label_tensor.unsqueeze(-1)
        val_label_tensor = val_label_tensor.unsqueeze(-1)
        test_label_tensor = test_label_tensor.unsqueeze(-1)  # 在最后新增一个维度，因为TensorDataset要第一维大小相同 label变为(batch,1)
        # mask矩阵的字典
#         train_batch_feature_tensor_history_mask_dict = generate_user_feature_alignment_tensor(train_list,train_data_tensor_hash_history_mask, is_mask=True)
#         val_batch_feature_tensor_history_mask_dict = generate_user_feature_alignment_tensor(val_list,val_data_tensor_hash_history_mask, is_mask=True)
#         test_batch_feature_tensor_history_mask_dict = generate_user_feature_alignment_tensor(test_list,test_data_tensor_hash_history_mask, is_mask=True)
        print('张量生成完成')
        
        # TensorDataset输入得是张量，因此由字典转为张量
        train_batch_feature_tensor_history_discrete = train_batch_feature_tensor_dict['history_discrete']
        train_batch_feature_tensor_history_continue = train_batch_feature_tensor_dict['history_continue']
#         train_batch_feature_tensor_history_discrete_mask = train_batch_feature_tensor_history_mask_dict['history_discrete']
#         train_batch_feature_tensor_history_continue_mask = train_batch_feature_tensor_history_mask_dict['history_continue']

        val_batch_feature_tensor_history_discrete = val_batch_feature_tensor_dict['history_discrete']
        val_batch_feature_tensor_history_continue = val_batch_feature_tensor_dict['history_continue']
#         val_batch_feature_tensor_history_discrete_mask = val_batch_feature_tensor_history_mask_dict['history_discrete']
#         val_batch_feature_tensor_history_continue_mask = val_batch_feature_tensor_history_mask_dict['history_continue']
        
        test_batch_feature_tensor_history_discrete = test_batch_feature_tensor_dict['history_discrete']
        test_batch_feature_tensor_history_continue = test_batch_feature_tensor_dict['history_continue']
#         test_batch_feature_tensor_history_discrete_mask = test_batch_feature_tensor_history_mask_dict['history_discrete']
#         test_batch_feature_tensor_history_continue_mask = test_batch_feature_tensor_history_mask_dict['history_continue']
#         print('val_label_tensor',val_label_tensor.shape,val_batch_feature_tensor_history_discrete.shape)
        # 训练集
#         print('train_dataset',train_batch_feature_tensor_history_discrete.shape,train_batch_feature_tensor_history_continue.shape,train_label_tensor.shape)
        train_dataset = TensorDataset(train_batch_feature_tensor_history_discrete,train_batch_feature_tensor_history_continue,
                                      train_label_tensor)
        val_dataset = TensorDataset(val_batch_feature_tensor_history_discrete,val_batch_feature_tensor_history_continue,
                                  val_label_tensor)

        # 旧
        # train_batch_feature_tensor = list(train_batch_feature_tensor_dict.values())
        # train_batch_feature_tensor_history_mask = list(train_batch_feature_tensor_history_mask_dict.values())
        # val_batch_feature_tensor = list(val_batch_feature_tensor_dict.values())
        # val_batch_feature_tensor_history_mask = list(val_batch_feature_tensor_history_mask_dict.values())
        # test_batch_feature_tensor = list(test_batch_feature_tensor_dict.values())
        # test_batch_feature_tensor_history_mask = list(test_batch_feature_tensor_history_mask_dict.values())  
        # # 训练集
        # train_dataset = TensorDataset(*train_batch_feature_tensor, *train_batch_feature_tensor_history_mask, train_label_tensor)
        # val_dataset = TensorDataset(*val_batch_feature_tensor, *val_batch_feature_tensor_history_mask, val_label_tensor)
        
        # 创建数据加载器
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, drop_last=True)  # 记得改回随机
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, drop_last=True)

        # 确保您的计算机上有CUDA支持的GPU
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        # 创建大模型的实例
        model = MatchingModel(feature_category_num_dict, feature_column_dict, continue_embedding_dim,
                    discrete_embedding_dim, num_heads, feature_dim, max_history_len)
        print('模型搭建完成')
        model.to(device)
        # 进一步处理 列表转移到GPU
#         for i in range(len(model.history_embedding_layer.history_discrete_embeddings)):
#             model.history_embedding_layer.history_discrete_embeddings[i] = \
#             model.history_embedding_layer.history_discrete_embeddings[i].to(device)
#         for i in range(len(model.history_embedding_layer.history_continue_embedding)):
#             model.history_embedding_layer.history_continue_embedding[i] = \
#             model.history_embedding_layer.history_continue_embedding[i].to(device)
        model.bilstm_layer = model.bilstm_layer.to(device)
        print('模型转移到GPU完成')
        lossfunction = nn.BCELoss()
    #     optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)
#         optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=lr, momentum=0.9)
        optimizer = optim.Adadelta(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)

        # 训练
        model_training(model, train_loader, val_loader, lossfunction, optimizer, 500, device)
        print('模型训练完成')
        print('||--------训练结束时间：',datetime.datetime.now(),'-------------')
        # 测试
        test_dataset = TensorDataset(test_batch_feature_tensor_history_discrete,test_batch_feature_tensor_history_continue,
                                      test_label_tensor)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=True)
        average_loss_test, average_auc_test,average_acc_test,average_f1_test,average_precision_test,average_recall_test = test_model(model, test_loader)
        # 测试的每个样本结果保存到csv
        # 将本次训练的结果添加到DataFrame中
        test_auc_df = test_auc_df.append({'时间':datetime.datetime.now(),'model':'BiLSTM','运行位置':'GPU','Type':'Abb_QOE','dataset':data_time_windows,'train_ratio':train_ratio,'feature_embedding':feature_dim,'batchSize':batch_size,'lr':lr,'max_history_len':max_history_len,'实验数': i + 1,'测试集总损失': average_loss_test, 'AUC': average_auc_test,'ACC': average_acc_test,'F1': average_f1_test,'Precision': average_precision_test,'Recall': average_recall_test}, ignore_index=True)
#         weight_result  = {'时间':datetime.datetime.now(),'model':'model3.1','运行位置':'GPU','Type':'Origin','dataset':data_time_windows,'train_ratio':train_ratio,'feature_embedding':feature_dim,'batchSize':batch_size,'lr':lr,'max_history_len':max_history_len,'实验数': i + 1,\
#                         'se_user_pay_QOE_weight':weight_result_dict['se_user_pay_QOE_weight'],'se_user_pay_CHONGHE_weight':weight_result_dict['se_user_pay_CHONGHE_weight'],\
#                         'se_user_pay_FUFEI_weight':weight_result_dict['se_user_pay_FUFEI_weight'],'se_target_QOE_weight':weight_result_dict['se_target_QOE_weight'],\
#                         'se_target_CHONGHE_weight':weight_result_dict['se_target_CHONGHE_weight'],'se_target_FUFEI_weight':weight_result_dict['se_target_FUFEI_weight'],\
#                         'target_history_pay_attention_QOE_weight':weight_result_dict['target_history_pay_attention_QOE_weight'],\
#                         'target_history_pay_attention_CHONGHE_weight':weight_result_dict['target_history_pay_attention_CHONGHE_weight'],\
#                         'target_history_pay_attention_FUFEI_weight':weight_result_dict['target_history_pay_attention_FUFEI_weight']}
#         test_weight_df = test_weight_df.append(weight_result, ignore_index=True)
    # 将结果保存到CSV文件中
    with open('./Dataset/maoerDL_result_maoer_pay_pred_model3_1.csv', 'a') as f:
        test_auc_df.to_csv(f, index=False)
#     with open('./Dataset/maoerDL_result_maoer_pay_pred_weight_model3_1.csv', 'a') as f:
#         test_weight_df.to_csv(f, index=False)
#     test_auc_df.to_csv('./Dataset/maoerDL_result_maoer_pay_pred_model3_1.csv', index=False)
#     test_weight_df.to_csv('./Dataset/maoerDL_result_maoer_pay_pred_weight_model3_1.csv', index=False)
    print('结果已输出')
    print('||--------当前时间窗',data_time_windows,'结束时间：',datetime.datetime.now(),'-------------')


i=:1
划分文件已存在，不再进行数据划分
数据预处理结束
数据划分完成
张量生成完成
模型搭建完成
模型转移到GPU完成
||--训练：---------- 1 个batch运行时间： 2024-03-18 19:22:43.004938 -------------
||--训练：---------- 2 个batch运行时间： 2024-03-18 19:22:43.101775 -------------
||--训练：---------- 3 个batch运行时间： 2024-03-18 19:22:43.193439 -------------
||--训练：---------- 4 个batch运行时间： 2024-03-18 19:22:43.285668 -------------
||--训练：---------- 5 个batch运行时间： 2024-03-18 19:22:43.382544 -------------
||--训练：---------- 6 个batch运行时间： 2024-03-18 19:22:43.481227 -------------
||--训练：---------- 7 个batch运行时间： 2024-03-18 19:22:43.580534 -------------
||--训练：---------- 8 个batch运行时间： 2024-03-18 19:22:43.680749 -------------
||--训练：---------- 9 个batch运行时间： 2024-03-18 19:22:43.780984 -------------
||--训练：---------- 10 个batch运行时间： 2024-03-18 19:22:43.881128 -------------
||--训练：---------- 11 个batch运行时间： 2024-03-18 19:22:43.980685 -------------
||--训练：---------- 12 个batch运行时间： 2024-03-18 19:22:44.078277 -------------
||--训练：---------- 13 个batch运行时间： 2024-03-18 19:22:44.178500

||--训练：---------- 6 个batch运行时间： 2024-03-18 19:22:54.128225 -------------
||--训练：---------- 7 个batch运行时间： 2024-03-18 19:22:54.221714 -------------
||--训练：---------- 8 个batch运行时间： 2024-03-18 19:22:54.310428 -------------
||--训练：---------- 9 个batch运行时间： 2024-03-18 19:22:54.399051 -------------
||--训练：---------- 10 个batch运行时间： 2024-03-18 19:22:54.487737 -------------
||--训练：---------- 11 个batch运行时间： 2024-03-18 19:22:54.575277 -------------
||--训练：---------- 12 个batch运行时间： 2024-03-18 19:22:54.663581 -------------
||--训练：---------- 13 个batch运行时间： 2024-03-18 19:22:54.751920 -------------
||--训练：---------- 14 个batch运行时间： 2024-03-18 19:22:54.839672 -------------
||--训练：---------- 15 个batch运行时间： 2024-03-18 19:22:54.929863 -------------
||--训练：---------- 16 个batch运行时间： 2024-03-18 19:22:55.017250 -------------
||--训练：---------- 17 个batch运行时间： 2024-03-18 19:22:55.104164 -------------
||--训练：---------- 18 个batch运行时间： 2024-03-18 19:22:55.194409 -------------
||--训练：---------- 19 个batch运行时间： 2024-03-1

||--训练：---------- 12 个batch运行时间： 2024-03-18 19:23:04.894437 -------------
||--训练：---------- 13 个batch运行时间： 2024-03-18 19:23:04.975388 -------------
||--训练：---------- 14 个batch运行时间： 2024-03-18 19:23:05.056177 -------------
||--训练：---------- 15 个batch运行时间： 2024-03-18 19:23:05.137096 -------------
||--训练：---------- 16 个batch运行时间： 2024-03-18 19:23:05.219696 -------------
||--训练：---------- 17 个batch运行时间： 2024-03-18 19:23:05.302781 -------------
||--训练：---------- 18 个batch运行时间： 2024-03-18 19:23:05.385544 -------------
||--训练：---------- 19 个batch运行时间： 2024-03-18 19:23:05.467539 -------------
||--训练：---------- 20 个batch运行时间： 2024-03-18 19:23:05.549359 -------------
||--训练：---------- 21 个batch运行时间： 2024-03-18 19:23:05.630147 -------------
||--训练：---------- 22 个batch运行时间： 2024-03-18 19:23:05.711816 -------------
||--训练：---------- 23 个batch运行时间： 2024-03-18 19:23:05.793389 -------------
||--训练：---------- 24 个batch运行时间： 2024-03-18 19:23:05.876971 -------------
||--训练：---------- 25 个batch运行时间： 2024-

||--训练：---------- 18 个batch运行时间： 2024-03-18 19:23:14.704454 -------------
||--训练：---------- 19 个batch运行时间： 2024-03-18 19:23:14.795375 -------------
||--训练：---------- 20 个batch运行时间： 2024-03-18 19:23:14.889825 -------------
||--训练：---------- 21 个batch运行时间： 2024-03-18 19:23:14.990670 -------------
||--训练：---------- 22 个batch运行时间： 2024-03-18 19:23:15.091405 -------------
||--训练：---------- 23 个batch运行时间： 2024-03-18 19:23:15.197099 -------------
||--训练：---------- 24 个batch运行时间： 2024-03-18 19:23:15.316914 -------------
||--训练：---------- 25 个batch运行时间： 2024-03-18 19:23:15.429322 -------------
||--训练：---------- 26 个batch运行时间： 2024-03-18 19:23:15.540498 -------------
||--训练：---------- 27 个batch运行时间： 2024-03-18 19:23:15.654466 -------------
||--训练：---------- 28 个batch运行时间： 2024-03-18 19:23:15.759946 -------------
||--训练：---------- 29 个batch运行时间： 2024-03-18 19:23:15.865644 -------------
||--训练：---------- 30 个batch运行时间： 2024-03-18 19:23:15.966705 -------------
||--训练：---------- 31 个batch运行时间： 2024-

||--训练：---------- 23 个batch运行时间： 2024-03-18 19:23:26.359970 -------------
||--训练：---------- 24 个batch运行时间： 2024-03-18 19:23:26.459539 -------------
||--训练：---------- 25 个batch运行时间： 2024-03-18 19:23:26.554704 -------------
||--训练：---------- 26 个batch运行时间： 2024-03-18 19:23:26.652487 -------------
||--训练：---------- 27 个batch运行时间： 2024-03-18 19:23:26.751055 -------------
||--训练：---------- 28 个batch运行时间： 2024-03-18 19:23:26.850350 -------------
||--训练：---------- 29 个batch运行时间： 2024-03-18 19:23:26.953496 -------------
||--训练：---------- 30 个batch运行时间： 2024-03-18 19:23:27.058484 -------------
||--训练：---------- 31 个batch运行时间： 2024-03-18 19:23:27.162730 -------------
||--训练：---------- 32 个batch运行时间： 2024-03-18 19:23:27.267654 -------------
||--训练：---------- 33 个batch运行时间： 2024-03-18 19:23:27.384980 -------------
||--训练：---------- 34 个batch运行时间： 2024-03-18 19:23:27.491464 -------------
||--训练：---------- 35 个batch运行时间： 2024-03-18 19:23:27.596292 -------------
||--训练：---------- 36 个batch运行时间： 2024-

...

||--训练：---------- 44 个batch运行时间： 2024-03-18 21:38:56.945748 -------------
||--训练：---------- 45 个batch运行时间： 2024-03-18 21:38:57.076560 -------------
||--训练：---------- 46 个batch运行时间： 2024-03-18 21:38:57.220628 -------------
||--训练：---------- 47 个batch运行时间： 2024-03-18 21:38:57.351866 -------------
||--训练：---------- 48 个batch运行时间： 2024-03-18 21:38:57.490917 -------------
||--训练：---------- 49 个batch运行时间： 2024-03-18 21:38:57.621365 -------------
||--训练：---------- 50 个batch运行时间： 2024-03-18 21:38:57.751596 -------------
||--训练：---------- 51 个batch运行时间： 2024-03-18 21:38:57.900435 -------------
||--训练：---------- 52 个batch运行时间： 2024-03-18 21:38:58.036621 -------------
||--训练：---------- 53 个batch运行时间： 2024-03-18 21:38:58.195138 -------------
||--训练：---------- 54 个batch运行时间： 2024-03-18 21:38:58.337796 -------------
||--训练：---------- 55 个batch运行时间： 2024-03-18 21:38:58.485019 -------------
||--训练：---------- 56 个batch运行时间： 2024-03-18 21:38:58.622138 -------------
||--训练：---------- 57 个batch运行时间： 2024-

||--训练：---------- 43 个batch运行时间： 2024-03-18 21:39:17.336824 -------------
||--训练：---------- 44 个batch运行时间： 2024-03-18 21:39:17.480764 -------------
||--训练：---------- 45 个batch运行时间： 2024-03-18 21:39:17.609179 -------------
||--训练：---------- 46 个batch运行时间： 2024-03-18 21:39:17.722275 -------------
||--训练：---------- 47 个batch运行时间： 2024-03-18 21:39:17.840038 -------------
||--训练：---------- 48 个batch运行时间： 2024-03-18 21:39:17.975220 -------------
||--训练：---------- 49 个batch运行时间： 2024-03-18 21:39:18.108749 -------------
||--训练：---------- 50 个batch运行时间： 2024-03-18 21:39:18.226444 -------------
||--训练：---------- 51 个batch运行时间： 2024-03-18 21:39:18.348666 -------------
||--训练：---------- 52 个batch运行时间： 2024-03-18 21:39:18.469291 -------------
||--训练：---------- 53 个batch运行时间： 2024-03-18 21:39:18.587691 -------------
||--训练：---------- 54 个batch运行时间： 2024-03-18 21:39:18.703973 -------------
||--训练：---------- 55 个batch运行时间： 2024-03-18 21:39:18.819645 -------------
||--训练：---------- 56 个batch运行时间： 2024-

||--训练：---------- 41 个batch运行时间： 2024-03-18 21:39:31.901265 -------------
||--训练：---------- 42 个batch运行时间： 2024-03-18 21:39:32.049985 -------------
||--训练：---------- 43 个batch运行时间： 2024-03-18 21:39:32.186357 -------------
||--训练：---------- 44 个batch运行时间： 2024-03-18 21:39:32.321929 -------------
||--训练：---------- 45 个batch运行时间： 2024-03-18 21:39:32.457397 -------------
||--训练：---------- 46 个batch运行时间： 2024-03-18 21:39:32.617115 -------------
||--训练：---------- 47 个batch运行时间： 2024-03-18 21:39:32.757855 -------------
||--训练：---------- 48 个batch运行时间： 2024-03-18 21:39:32.895604 -------------
||--训练：---------- 49 个batch运行时间： 2024-03-18 21:39:33.031281 -------------
||--训练：---------- 50 个batch运行时间： 2024-03-18 21:39:33.191497 -------------
||--训练：---------- 51 个batch运行时间： 2024-03-18 21:39:33.330542 -------------
||--训练：---------- 52 个batch运行时间： 2024-03-18 21:39:33.468597 -------------
||--训练：---------- 53 个batch运行时间： 2024-03-18 21:39:33.613193 -------------
||--训练：---------- 54 个batch运行时间： 2024-

||--训练：---------- 40 个batch运行时间： 2024-03-18 21:39:47.854246 -------------
||--训练：---------- 41 个batch运行时间： 2024-03-18 21:39:47.997217 -------------
||--训练：---------- 42 个batch运行时间： 2024-03-18 21:39:48.126439 -------------
||--训练：---------- 43 个batch运行时间： 2024-03-18 21:39:48.256321 -------------
||--训练：---------- 44 个batch运行时间： 2024-03-18 21:39:48.394729 -------------
||--训练：---------- 45 个batch运行时间： 2024-03-18 21:39:48.530543 -------------
||--训练：---------- 46 个batch运行时间： 2024-03-18 21:39:48.661106 -------------
||--训练：---------- 47 个batch运行时间： 2024-03-18 21:39:48.790294 -------------
||--训练：---------- 48 个batch运行时间： 2024-03-18 21:39:48.929006 -------------
||--训练：---------- 49 个batch运行时间： 2024-03-18 21:39:49.061991 -------------
||--训练：---------- 50 个batch运行时间： 2024-03-18 21:39:49.189651 -------------
||--训练：---------- 51 个batch运行时间： 2024-03-18 21:39:49.329097 -------------
||--训练：---------- 52 个batch运行时间： 2024-03-18 21:39:49.455387 -------------
||--训练：---------- 53 个batch运行时间： 2024-

||--测试：---------- 7 个batch运行时间： 2024-03-18 21:39:58.398158 -------------
||--测试：---------- 8 个batch运行时间： 2024-03-18 21:39:58.413226 -------------
||--测试：---------- 9 个batch运行时间： 2024-03-18 21:39:58.428354 -------------
||--测试：---------- 10 个batch运行时间： 2024-03-18 21:39:58.446917 -------------
||--测试：---------- 11 个batch运行时间： 2024-03-18 21:39:58.461864 -------------
||--测试：---------- 12 个batch运行时间： 2024-03-18 21:39:58.476410 -------------
||--测试：---------- 13 个batch运行时间： 2024-03-18 21:39:58.491231 -------------
||--测试：---------- 14 个batch运行时间： 2024-03-18 21:39:58.505774 -------------
||--测试：---------- 15 个batch运行时间： 2024-03-18 21:39:58.521188 -------------
||--测试：---------- 16 个batch运行时间： 2024-03-18 21:39:58.535769 -------------
||--测试：---------- 17 个batch运行时间： 2024-03-18 21:39:58.552033 -------------
||--测试：---------- 18 个batch运行时间： 2024-03-18 21:39:58.566878 -------------
||--测试：---------- 19 个batch运行时间： 2024-03-18 21:39:58.581985 -------------
||--测试：---------- 20 个batch运行时间： 2024-03-